**Goal:** represent the tracks downloaded from [Wikiloc](http://wikiloc.com/) or transformed into .gpx format from Garmin .fit files, using for example the [gpsbabel](http://www.gpsbabel.org/) tool.

In [1]:
import re
import os
import folium
import glob
import logging
import numpy as np
from geopy.geocoders import Nominatim
from folium import plugins
import matplotlib.pyplot as plt
from matplotlib import colors

In [2]:
print folium.__version__

0.2.0


# User inputs

Define the directory where the files are stored and where the html will be written.

In [3]:
gpxdir = '/home/ctroupin/DataOceano/GPX/ComparisonTeix/'
outputdir = '/home/ctroupin/public_html/LeafletMaps'
mapname = '20160811_comparison_Teix.html'
gpxbasename = '*.gpx'
gpxfilelist = sorted(glob.glob(os.path.join(gpxdir, gpxbasename)))
colorlist = ['red', 'blue', 'green']
namelist = ['Suunto', 'Garmin', 'Wikiloc']
iconlist = ['https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Suunto-logo.svg/500px-Suunto-logo.svg.png',
            'https://upload.wikimedia.org/wikipedia/commons/thumb/c/c7/Garmin_logo.svg/640px-Garmin_logo.svg.png',
            'https://upload.wikimedia.org/wikipedia/commons/c/c2/Wikiloc-logo-big_2x.png']

Initialise the map

In [22]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='/home/ctroupin/logs/GPXheadmap.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

geolocator = Nominatim()
location = geolocator.geocode({"city":"Valldemossa", "country": "Spain"}, exactly_one=True)
logger.info("Working on location: %s" %(location[0]))

INFO:root:Working on location: Valldemossa, Serra de Tramuntana, Illes Balears, España


Function to read GPX

In [5]:
def read_gpx(filename):
    coords = []
    with open(filename) as f:
        for lines in f:
            match = re.search('<trkpt lat="([-0-9\.]+)" lon="([-0-9\.]+)">', lines)
            if match:
                coords.append((float(match.group(1)), float(match.group(2))))
    return coords

Loop on the files

In [31]:
logger.info('Creating a new map')
map_run = folium.Map(location=[39.7261348, 2.641662], 
                     zoom_start=15)

logger.info('Loop on the GPX files')
for gpxfiles, color, name, iconurl in zip(gpxfilelist, colorlist, namelist, iconlist):
    logger.debug(gpxfiles)
    coords = read_gpx(gpxfiles)
    icon = folium.features.CustomIcon(iconurl, icon_size=(50, 20))
    folium.Marker(coords[0], popup=name, icon=icon).add_to(map_run)
    folium.PolyLine(coords, color=color, weight=4).add_to(map_run)
    
map_run

INFO:root:Creating a new map
INFO:root:Loop on the GPX files
DEBUG:root:/home/ctroupin/DataOceano/GPX/ComparisonTeix/Teix_Garmin.gpx
DEBUG:root:/home/ctroupin/DataOceano/GPX/ComparisonTeix/Teix_Suunto.gpx
DEBUG:root:/home/ctroupin/DataOceano/GPX/ComparisonTeix/Teix_Wikiloc.gpx


In [32]:
if not(os.path.exists(outputdir)):
    os.makedirs(outputdir)
    logging.info('Creating directory %s' %(outputdir))
map_run.save(os.path.join(outputdir, mapname))